In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect



# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [2]:
#books_df = pd.read_csv("orignal_data/bestsellers_with_categories.csv")
books_df = pd.read_csv("static/data/bestsellers_with_categories.csv")
books_df

,Name,Author,User_Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction
...,...,...,...,...,...,...,...
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction


In [3]:
books_df.count()

Name           550
Author         550
User_Rating    550
Reviews        550
Price          550
Year           550
Genre          550
dtype: int64

In [4]:
books_df = books_df.rename(columns={'User Rating' : 'User_Rating'})
books_df

,Name,Author,User_Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction
...,...,...,...,...,...,...,...
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction


In [5]:
# Create book class 
# ----------------------------------

# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

# Creates Classes which will serve as the anchor points for our Tables
class Book(Base):
    __tablename__ = 'books'
    Id = Column(Integer, primary_key=True)
    Name = Column(String(255))
    Author = Column(String(255))
    User_Rating = Column(Float)
    Reviews = Column(Integer)
    Price = Column(Float)
    Year = Column(Integer)
    Genre = Column(String(255))

In [6]:
# Create engine
engine = create_engine('sqlite:///amazon_books.sqlite')
sqlite_connection = engine.connect()

In [7]:
# Use this to clear out the db
# ----------------------------------
Base.metadata.drop_all(engine)

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [8]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

# Base.classes.keys()
# reflect an existing database into a new model
Base1 = automap_base()
# reflect the tables
Base1.prepare(engine, reflect=True)

# Save reference to the table
# Books = Base.classes.books
Base1.classes.keys()

['books']

In [9]:
objects = [];
for index,row in books_df.iterrows():
    objects.append(Book(Name=row['Name'],Author=row['Author'],User_Rating=row['User_Rating'],
                        Reviews=row['Reviews'],Price=row['Price'],Year=row['Year'],Genre=row['Genre']))


In [10]:
session.bulk_save_objects(objects)
session.commit()

In [11]:
# Save reference to the table
Books = Base1.classes.books

In [12]:
# results = session.query().all()
# results

for row in session.query(Books).all():
    #print(vars(row))
    print(vars(row))



{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000002773C0E6780>, 'Genre': 'Non Fiction', 'Price': 8.0, 'User_Rating': 4.7, 'Name': '10-Day Green Smoothie Cleanse', 'Id': 1, 'Year': 2016, 'Reviews': 17350, 'Author': 'JJ Smith'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000002773C0E67F0>, 'Genre': 'Fiction', 'Price': 22.0, 'User_Rating': 4.6, 'Name': '11/22/63: A Novel', 'Id': 2, 'Year': 2011, 'Reviews': 2052, 'Author': 'Stephen King'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000002773C0E6860>, 'Genre': 'Non Fiction', 'Price': 15.0, 'User_Rating': 4.7, 'Name': '12 Rules for Life: An Antidote to Chaos', 'Id': 3, 'Year': 2018, 'Reviews': 18979, 'Author': 'Jordan B. Peterson'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000002773C0E6908>, 'Genre': 'Fiction', 'Price': 6.0, 'User_Rating': 4.7, 'Name': '1984 (Signet Classics)', 'Id': 4, 'Year': 2017, 'Reviews': 21424, 'Author': 'Georg

In [15]:
result = session.query(Books).all()
result

In [18]:
count = 0;
for row in result:
    count= count+1
    print(row.Name)
    

10-Day Green Smoothie Cleanse
11/22/63: A Novel
12 Rules for Life: An Antidote to Chaos
1984 (Signet Classics)
5,000 Awesome Facts (About Everything!) (National Geographic Kids)
A Dance with Dragons (A Song of Ice and Fire)
A Game of Thrones / A Clash of Kings / A Storm of Swords / A Feast of Crows / A Dance with Dragons
A Gentleman in Moscow: A Novel
A Higher Loyalty: Truth, Lies, and Leadership
A Man Called Ove: A Novel
A Man Called Ove: A Novel
A Patriot's History of the United States: From Columbus's Great Discovery to the War on Terror
A Stolen Life: A Memoir
A Wrinkle in Time (Time Quintet)
Act Like a Lady, Think Like a Man: What Men Really Think About Love, Relationships, Intimacy, and Commitment
Adult Coloring Book Designs: Stress Relief Coloring Book: Garden Designs, Mandalas, Animals, and Paisley Patterns
Adult Coloring Book: Stress Relieving Animal Designs
Adult Coloring Book: Stress Relieving Patterns
Adult Coloring Books: A Coloring Book for Adults Featuring Mandalas and H

In [19]:
count

550